In [ ]:
import ntpath, os
import azureml.core, azureml.data
print("SDK version:", azureml.core.VERSION)

from azureml.contrib.pipeline.steps import ParallelRunStep, ParallelRunConfig
from azureml.core import Datastore, Environment, Experiment, ScriptRunConfig, Workspace
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.dataset import Dataset
from azureml.core.model import Model
from azureml.core.runconfig import DEFAULT_CPU_IMAGE, RunConfiguration
from azureml.data.azure_storage_datastore import AzureFileDatastore, AzureBlobDatastore
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails
from pathlib import Path

In [ ]:
DEFAULT_CPU_IMAGE

In [ ]:
%run Common.ipynb

In [ ]:
ws = Workspace.from_config()
print('Name: {0}'.format(ws.name), 'Resource Group: {0}'.format(ws.resource_group), 'Location: {0}'.format(ws.location), 'Subscription Id: {0}'.format(ws.subscription_id), sep = '\n')

In [ ]:
compute_name = 'CPU'

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
else:
    provisioning_configuration = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2',
                                                                min_nodes = 1,
                                                                max_nodes = 2)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_configuration)
    
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
    print(compute_target.status.serialize())

In [ ]:
compute_name = 'GPU'

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
else:
    provisioning_configuration = AmlCompute.provisioning_configuration(vm_size = 'Standard_NC6',
                                                                min_nodes = 1,
                                                                max_nodes = 2)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_configuration)
    
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
    print(compute_target.status.serialize())

In [ ]:
DATA_DIR = './data/CamVid/'

# load repo with data if it is not exists
if not os.path.exists(DATA_DIR):
    print('Loading data...')
    os.system('git clone https://github.com/alexgkendall/SegNet-Tutorial ./data')
    print('Done!')

In [ ]:
raw = Datastore.register_azure_blob_container(workspace=ws,
                                              datastore_name='raw',
                                              container_name='raw',
                                              account_name='healthyhabitatsvb',
                                              account_key='0crB6HIwJxefIX/Op19ZVLepgE41jCp/SYTnlBGpPBB0To9Jrpf37pLvWX/eMhNyIVM8JxBC+n94IQOH1Cf9/g==',
                                              create_if_not_exists=True)

In [ ]:
target_path = "CamVid"

In [ ]:
raw.upload(src_dir=DATA_DIR[0:-1],
                 target_path=target_path,
                 overwrite=True,
                 show_progress=True)

In [ ]:
camvid_data = DataReference(
    datastore=raw,
    data_reference_name='camvid_data',
    path_on_datastore=target_path)

#### Scripts
- train.py
- score.py

In [ ]:
%%writefile ./scripts/train.py

import argparse, cv2, keras, os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import albumentations as A
import matplotlib.pyplot as plt
import numpy as np
import segmentation_models as sm

from azureml.core import Run
from keras.callbacks import Callback

print("In train.py")

parser = argparse.ArgumentParser("train")

parser.add_argument("--input_data", type=str, help="input data")

args = parser.parse_args()

print("Argument 1: %s" % args.input_data)

x_train_dir = os.path.join(args.input_data, 'train')
y_train_dir = os.path.join(args.input_data, 'trainannot')

x_valid_dir = os.path.join(args.input_data, 'val')
y_valid_dir = os.path.join(args.input_data, 'valannot')

x_test_dir = os.path.join(args.input_data, 'test')
y_test_dir = os.path.join(args.input_data, 'testannot')

# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
    plt.savefig('{0}/{1}.png'.format(args.input_data, name))
    plt.savefig('./outputs/{0}.png'.format(name))
    
    
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x
    

# classes for data loading and preprocessing
class Dataset:
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    CLASSES = ['sky', 'building', 'pole', 'road', 'pavement', 
               'tree', 'signsymbol', 'fence', 'car', 
               'pedestrian', 'bicyclist', 'unlabelled']
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i], 0)
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        # add background if mask is not binary
        if mask.shape[-1] != 1:
            background = 1 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        return len(self.ids)
    
    
class Dataloder(keras.utils.Sequence):
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):
        
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
        
        # transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        
        return batch
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)

def round_clip_0_1(x, **kwargs):
    return x.round().clip(0, 1)

# define heavy augmentations
def get_training_augmentation():
    train_transform = [

        A.HorizontalFlip(p=0.5),

        A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        A.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
        A.RandomCrop(height=320, width=320, always_apply=True),

        A.IAAAdditiveGaussianNoise(p=0.2),
        A.IAAPerspective(p=0.5),

        A.OneOf(
            [
                A.CLAHE(p=1),
                A.RandomBrightness(p=1),
                A.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        A.OneOf(
            [
                A.IAASharpen(p=1),
                A.Blur(blur_limit=3, p=1),
                A.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        A.OneOf(
            [
                A.RandomContrast(p=1),
                A.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
        A.Lambda(mask=round_clip_0_1)
    ]
    return A.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        A.PadIfNeeded(384, 480)
    ]
    return A.Compose(test_transform)

def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        A.Lambda(image=preprocessing_fn),
    ]
    return A.Compose(_transform)

# Lets look at data we have
dataset = Dataset(x_train_dir, y_train_dir, classes=['car', 'pedestrian'])

image, mask = dataset[5] # get some sample
visualize(
    image=image, 
    cars_mask=mask[..., 0].squeeze(),
    sky_mask=mask[..., 1].squeeze(),
    background_mask=mask[..., 2].squeeze(),
)

# Lets look at augmented data we have
dataset = Dataset(x_train_dir, y_train_dir, classes=['car', 'sky'], augmentation=get_training_augmentation())

image, mask = dataset[12] # get some sample
visualize(
    image=image, 
    cars_mask=mask[..., 0].squeeze(),
    sky_mask=mask[..., 1].squeeze(),
    background_mask=mask[..., 2].squeeze(),
)

BACKBONE = 'efficientnetb3'
BATCH_SIZE = 8
CLASSES = ['car']
LR = 0.0001
#EPOCHS = 40
EPOCHS = 5

preprocess_input = sm.get_preprocessing(BACKBONE)

# define network parameters
n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

# define optomizer
optim = keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

# Dataset for train images
train_dataset = Dataset(
    x_train_dir, 
    y_train_dir, 
    classes=CLASSES, 
    augmentation=get_training_augmentation(),
    preprocessing=get_preprocessing(preprocess_input),
)

# Dataset for validation images
valid_dataset = Dataset(
    x_valid_dir, 
    y_valid_dir, 
    classes=CLASSES, 
    augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocess_input),
)

train_dataloader = Dataloder(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = Dataloder(valid_dataset, batch_size=1, shuffle=False)

# check shapes for errors
assert train_dataloader[0][0].shape == (BATCH_SIZE, 320, 320, 3)
assert train_dataloader[0][1].shape == (BATCH_SIZE, 320, 320, n_classes)

run = Run.get_context()

class LogRunMetrics(Callback):
    # callback at the end of every epoch
    def on_epoch_end(self, epoch, log):
        # log a value repeated which creates a list
        run.log('Val. Loss', log['val_loss'])
        run.log('Val. IoU Score', log['val_iou_score'])
        run.log('Val. F1 Score', log['val_f1-score'])
        run.log('Loss', log['loss'])
        run.log('IoU Score', log['iou_score'])
        run.log('F1 Score', log['f1-score'])
        run.log('LR', log['lr'])

# define callbacks for learning rate scheduling and best checkpoints saving
callbacks = [
    keras.callbacks.ModelCheckpoint('./outputs/best_model.h5', save_weights_only=True, save_best_only=True, mode='min'),
    #keras.callbacks.ModelCheckpoint(os.path.join(args.input_data, 'best_model.h5'), save_weights_only=True, save_best_only=True, mode='min'),
    keras.callbacks.ReduceLROnPlateau(),
    LogRunMetrics()
]

# train model
history = model.fit_generator(
    train_dataloader, 
    steps_per_epoch=len(train_dataloader), 
    epochs=EPOCHS, 
    callbacks=callbacks, 
    validation_data=valid_dataloader, 
    validation_steps=len(valid_dataloader),
)

fname = 'plot.jpg'

# Plot training & validation iou_score values
plt.figure(figsize=(30, 5))
plt.subplot(121)
plt.plot(history.history['iou_score'])
plt.plot(history.history['val_iou_score'])
plt.title('Model iou_score')
plt.ylabel('iou_score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('{0}/{1}'.format(args.input_data, fname))
plt.savefig('./outputs/{0}'.format(fname))

test_dataset = Dataset(
    x_test_dir,
    y_test_dir,
    classes=CLASSES,
    augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocess_input),
)

test_dataloader = Dataloder(test_dataset, batch_size=1, shuffle=False)

# load best weights
model.load_weights('./outputs/best_model.h5') 

scores = model.evaluate_generator(test_dataloader)

print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric.__name__, value))

n = 5
ids = np.random.choice(np.arange(len(test_dataset)), size=n)

for i in ids:
    
    image, gt_mask = test_dataset[i]
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image).round()
    
    visualize(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[..., 0].squeeze(),
        pr_mask=pr_mask[..., 0].squeeze(),
    )
    
model = run.register_model(model_name='qubvel-segmentation_models-u-net', model_path='./outputs/best_model.h5')
print(model.name, model.id, model.version, sep = '\t')

In [ ]:
!python3.6 -m pip install azureml-contrib-pipeline-steps

In [ ]:
import keras, os, sys

import numpy as np
import segmentation_models as sm

from azureml.core import Run
from azureml.core.model import Model

model_path = Model.get_model_path('qubvel-segmentation_models-u-net', _workspace=ws)
print(model_path)
BACKBONE = 'efficientnetb3'
CLASSES = ['car']
LR = 0.0001

# define network parameters
n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

# define optomizer
optim = keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

model.load_weights(model_path)

In [ ]:
%%writefile ./scripts/score.py

import keras, os, sys

import numpy as np
import segmentation_models as sm

from azureml.core import Run
from azureml.core.model import Model

print("In score.py")

def init():
    global model
    
    parser = argparse.ArgumentParser("score")
    
    parser.add_argument('--model_name', type=str, help="model name", required=True)
    
    args, unknown_args = parser.parse_known_args()

    model_path = Model.get_model_path(args.model_name)
    print(model_path)
    
    print(os.path.isfile(model_path))
    
    BACKBONE = 'efficientnetb3'
    CLASSES = ['car']
    LR = 0.0001

    # define network parameters
    n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
    activation = 'sigmoid' if n_classes == 1 else 'softmax'

    #create model
    model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)
    
    # define optomizer
    optim = keras.optimizers.Adam(LR)
    
    # Segmentation models losses can be combined together by '+' and scaled by integer or float factor
    dice_loss = sm.losses.DiceLoss()
    focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
    total_loss = dice_loss + (1 * focal_loss)

    # actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
    # total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

    # compile keras model with defined optimozer, loss and metrics
    model.compile(optim, total_loss, metrics)

    model.load_weights(model_path)
    
def run(mini_batch):
    resultList = []
    print(mini_batch)
    
    '''
    test_dataloader = Dataloder(test_dataset, batch_size=1, shuffle=False)

    scores = model.evaluate_generator(test_dataloader)

    print("Loss: {:.5}".format(scores[0]))
    for metric, value in zip(metrics, scores[1:]):
        print("mean {}: {:.5}".format(metric.__name__, value))
    '''
    
    '''
    for image in mini_batch:
        # prepare each image
        data = Image.open(image)
        np_im = np.array(data).reshape((1, 784))
        # perform inference
        inference_result = output.eval(feed_dict={in_tensor: np_im}, session=g_tf_sess)
        # find best probability, and add to result list
        best_result = np.argmax(inference_result)
        resultList.append("{}: {}".format(os.path.basename(image), best_result))
    '''
    return resultList

### Train Model Pipeline

In [ ]:
conda_dependencies = CondaDependencies()
conda_dependencies.add_pip_package('albumentations')
conda_dependencies.add_pip_package('keras')
conda_dependencies.add_pip_package('matplotlib')
conda_dependencies.add_pip_package('opencv-python')
conda_dependencies.add_pip_package('segmentation-models')
conda_dependencies.add_pip_package('tensorflow')

environment = Environment(name='qubvel-segmentation_models')
environment.docker.enabled = True
environment.docker.base_image = DEFAULT_CPU_IMAGE
environment.python.conda_dependencies = conda_dependencies
environment.python.user_managed_dependencies = False

run_configuration = RunConfiguration()
run_configuration.environment = environment
run_configuration.target = compute_target

In [ ]:
source_directory = './scripts'

train_step = PythonScriptStep(name='train',
                              source_directory=source_directory,
                              script_name='train.py',
                              arguments=['--input_data', camvid_data],
                              inputs=[camvid_data],
                              compute_target=compute_target,
                              runconfig=run_configuration,
                              allow_reuse=False)

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[train_step])

In [ ]:
experiment = Experiment(ws, 'train_pipeline')
experiment

In [ ]:
pipeline_run = experiment.submit(pipeline)

In [ ]:
RunDetails(pipeline_run).show()

In [ ]:
pipeline_run.wait_for_completion(show_output=True)

### Batch Score Pipeline

In [ ]:
latest_model = None
model_version = 0

models = Model.list(ws)

for model in models:
    if model.name == 'qubvel-segmentation_models-u-net':
        if model.version > model_version:
            model_version = model.version
            latest_model = model

In [ ]:
latest_model

In [ ]:
dataset_name = 'camvid_dataset'

camvid_dataset = Dataset.File.from_files(path=camvid_data , validate=False)
registered_camvid_dataset = camvid_dataset.register(ws, dataset_name, create_new_version=True)
named_camvid_dataset = registered_camvid_dataset.as_named_input(dataset_name)

In [ ]:
output_data = PipelineData(name="results",
                           datastore=raw,
                           output_path_on_compute="CamVid/results")

In [ ]:
source_directory = './scripts'

parallel_run_config = ParallelRunConfig(source_directory=source_directory,
                                        entry_script='score.py',
                                        mini_batch_size='5',
                                        error_threshold=10,
                                        output_action='append_row',
                                        environment=environment,
                                        compute_target=compute_target,
                                        node_count=2)

In [ ]:
parallel_run_step = ParallelRunStep(name='batch-score',
                                    parallel_run_config=parallel_run_config,
                                    arguments=['--model_name', 'qubvel-segmentation_models-u-net'],
                                    inputs=[named_camvid_dataset],
                                    output=output_data,
                                    models=[],
                                    allow_reuse=False)

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[parallel_run_step])

In [ ]:
experiment = Experiment(ws, 'batch_score_pipeline')
experiment

In [ ]:
pipeline_run = experiment.submit(pipeline)

In [ ]:
RunDetails(pipeline_run).show()

In [ ]:
pipeline_run.wait_for_completion(show_output=True)